In [1]:
import chromadb
import torch
from sklearn.metrics import accuracy_score

In [2]:

chroma_client = chromadb.PersistentClient(path="./embedding_data/embedding_store")
collection_names = ["pretrained_resnet18_embeddings", "custom_resnet18_embeddings", "combined_loss_training", "fixed_centroids_training"]
# Adjust index to run notebook on different embeddings
collection = chroma_client.get_collection(name=collection_names[3])


In [5]:
test_dataset = collection.get(
    include=['embeddings', 'documents', 'metadatas'],
    where = {"dataset_type": "test"}
)
train_dataset = collection.get(
    include=['embeddings', 'documents', 'metadatas'],
    where = {"dataset_type": "train"}
)

In [6]:
top_10_nn = collection.query(
    query_embeddings=test_dataset['embeddings'][0],
    where={"dataset_type": "train"}
)
knn_labels = [x['label']for x in top_10_nn['metadatas'][0]]

In [7]:
y = []
predictions = []
count = 0
for test_id, test_embedding, test_meta in list(zip(test_dataset['ids'], test_dataset['embeddings'], test_dataset['metadatas'])):
    if count % 100 == 0:
        print(count)
    count += 1
    test_label = test_meta['label']
    y.append(test_label)
    top_10_nn = collection.query(
        query_embeddings=test_embedding,
        where={"dataset_type": "train"}
    )
    knn_labels = [x['label']for x in top_10_nn['metadatas'][0]]
    prediction = max(set(knn_labels), key=knn_labels.count)
    predictions.append(prediction)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [8]:
accuracy_score(y, predictions)

0.2516